# Project 4, Git Practice, APS1070 Fall 2022
#### **Linear Regression -  14 points**
**Deadline: Dec 1st, 21:00**

**Academic Integrity**

This project is individual - it is to be completed on your own. If you have questions, please post your query in the APS1070 Piazza Q&A forums (the answer might be useful to others!).

Do not share your code with others, or post your work online. Do not submit code that you have not written yourself. Students suspected of plagiarism on a project, midterm or exam will be referred to the department for formal discipline for breaches of the Student Code of Conduct.

Please fill out the following:


*   Your **name**: Tongfei Li
*   Your **student number**: 1004759460



## Gradient Descent with Additional Features and Regularization [2 marks + 1 mark Git submission]

We'll apply linear regresssion to a toy dataset (`LR_data.csv`), with 10 features `x1`-`x10` and a "measurement" `y`. We'll take a few shortcuts by using built-in sklearn functions.

1. Data Preparation **[0.5]**
  * Print the dataset, and create Numpy arrays with inputs (X) and outputs (y). 
  * Split the dataset into training and validation sets (80% training, 20% validation). When splitting, set `random_state=1`.

2. Linear Regression **[0.5]**
  * Standardize the data using StandardScaler from sklearn.
  * Use the `sklearn.linear_model.LinearRegression` function [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) to perform linear regression.
  * Print the RMSE for training and validation data.

3. Linear Regression with Additional Features **[0.5]**
  * Let's add more features to our dataset (degree 8) using `sklearn.preprocessing.PolynomialFeatures` [here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html). You'll want to create the additional features first, then perform standardization (start from non-standardized data).
  * Again, use `sklearn.linear_model.LinearRegression` to perform linear regression.
  * Print the RMSE for training and validation data.

4. Linear Regression with Additional Features and Regularization **[0.5]**
  * Let's switch models, and instead use the `sklearn.linear_model.Ridge` function [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) to perform linear regression with regularization. Apply the model to the processed data (additional, standardized) you used in 3 above. Use a `FOR` loop to run `sklearn.linear_model.Ridge` with different `alpha` values. Specifically, sweep `alpha` from 1E-2 to 1E10 (each step is an order of magnitude jump).

In [1]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/aps1070-2019/datasets/master/LR_data.csv" , skipinitialspace=True)

In [2]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import linalg
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

### 1. Data Preparation **[0.5]**

  * Print the dataset, and create Numpy arrays with inputs (X) and outputs (y).

In [3]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,y
0,1.454481,4.940543,7.116015,3.805131,4.105432,1.832290,5.211909,5.088293,3.841250,6.121133,6.575066e+06
1,4.907182,1.653654,7.339445,9.065282,7.169233,2.354136,2.893005,8.720131,8.646473,3.806860,8.419881e+06
2,1.842804,1.183878,4.203206,6.815847,6.655899,8.295816,3.556779,5.794679,3.260723,5.463808,9.745537e+04
3,2.393457,0.094127,3.208929,0.497443,4.318127,4.999247,9.470238,3.061990,7.312995,5.475240,1.129200e+04
4,5.093724,3.905293,4.858562,4.465484,0.947091,9.410307,8.137329,9.043838,9.128607,0.790795,3.105448e+05
...,...,...,...,...,...,...,...,...,...,...,...
195,2.936450,2.094812,9.493814,3.411240,4.350940,0.386062,1.303290,2.699563,7.622569,3.037162,6.599726e+07
196,4.915597,1.953974,7.312552,9.331203,1.329366,8.286760,5.233018,7.476923,7.043560,5.745565,8.176250e+06
197,6.772096,6.406489,4.638352,7.042340,0.912338,7.568296,1.194928,2.592840,9.386478,4.147710,2.144147e+05
198,3.940339,4.330498,6.693371,9.169265,0.858846,0.601942,2.635455,2.037859,7.883720,8.525164,4.028735e+06


In [4]:
df.drop(columns=['y']).values

array([[1.45448096, 4.94054287, 7.11601489, ..., 5.08829321, 3.84125023,
        6.12113346],
       [4.90718225, 1.65365381, 7.33944456, ..., 8.72013079, 8.64647279,
        3.80685964],
       [1.84280389, 1.18387834, 4.20320584, ..., 5.79467869, 3.26072251,
        5.46380776],
       ...,
       [6.77209621, 6.40648915, 4.6383525 , ..., 2.59284   , 9.38647814,
        4.14770963],
       [3.94033913, 4.3304978 , 6.69337057, ..., 2.03785905, 7.88372005,
        8.52516391],
       [8.63612874, 1.27293914, 4.33917484, ..., 4.91796749, 9.54377194,
        5.67386964]])

In [6]:
X = df.drop(columns=['y']).values
y = df.y.values.reshape(-1,1)

In [27]:
X.shape

(200, 10)

  * Split the dataset into training and validation sets (80% training, 20% validation). When splitting, set random_state=1.

In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=1)


### 2. Linear Regression **[0.5]**

  * Standardize the data using StandardScaler from sklearn.

In [8]:
scaler = StandardScaler()
Xs_train = scaler.fit_transform(X_train)
Xs_val = scaler.transform(X_val)
Xs_train

array([[ 1.3003838 ,  1.09240598, -1.10715222, ...,  0.17730494,
         0.29239242, -0.10062555],
       [-0.65136938, -1.05979053, -0.3780466 , ...,  1.43349113,
        -0.35954867, -0.38347151],
       [ 0.63491406, -1.78554978,  0.45750412, ...,  0.01295335,
        -1.27398115,  0.07054911],
       ...,
       [ 1.7833874 ,  1.31178965,  0.79657848, ..., -0.10949835,
         0.83961754, -0.74776912],
       [ 0.93207799, -0.76735554, -0.92633629, ..., -0.43743407,
         1.14550533,  0.50510919],
       [-0.73214424, -1.11212895,  1.74018506, ..., -0.14082142,
         1.21944727,  1.3074281 ]])

  * Use the `sklearn.linear_model.LinearRegression` function to perform linear regression.

In [9]:
reg = LinearRegression().fit(Xs_train, y_train)
#reg.coef_
reg.score(Xs_train, y_train)

0.5190965814802768

  * Print the RMSE for training and validation data.

In [10]:
# function defined to calculate rmse
def rmse(yPred, y):
    return np.sqrt(mean_squared_error(yPred, y))

In [11]:
yp_train = reg.predict(Xs_train)
yp_val = reg.predict(Xs_val)
print('RMSE for training data:', rmse(yp_train,y_train))
print('RMSE for validation data:', rmse(yp_val,y_val))

RMSE for training data: 16296980.655667374
RMSE for validation data: 14061578.864980102


3. Linear Regression with Additional Features **[0.5]**
  * Let's add more features to our dataset (degree 8) using `sklearn.preprocessing.PolynomialFeatures` You'll want to create the additional features first, then perform standardization (start from non-standardized data).
  

In [39]:
poly = PolynomialFeatures(8)
# start from non-standardized data
X_poly_train = poly.fit_transform(X_train)
X_poly_val = poly.transform(X_val)
# standardize data
scaler = StandardScaler()
Xs_poly_train = scaler.fit_transform(X_poly_train)
Xs_poly_val = scaler.transform(X_poly_val)
Xs_poly_train.shape

(160, 43758)

* Again, use `sklearn.linear_model.LinearRegression` to perform linear regression.

In [40]:
# apply linear regression
reg_poly_st = LinearRegression().fit(Xs_poly_train, y_train)
print(reg_poly_st.score(Xs_poly_train, y_train))

1.0


* Print the RMSE for training and validation data.

In [42]:
yps_poly_train = reg_poly_st.predict(Xs_poly_train)
yps_poly_val = reg_poly_st.predict(Xs_poly_val)
print('----- Report RMSE ------')
print('RMSE for training data:', rmse(yps_poly_train,y_train))
print('RMSE for validation data:', rmse(yps_poly_val,y_val))

----- Report RMSE ------
RMSE for training data: 2.3663448165888318e-07
RMSE for training data: 10912736.457461182


4. Linear Regression with Additional Features and Regularization **[0.5]**
  * Let's switch models, and instead use the `sklearn.linear_model.Ridge` function [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) to perform linear regression with regularization. Apply the model to the processed data (additional, standardized) you used in 3 above. Use a `FOR` loop to run `sklearn.linear_model.Ridge` with different `alpha` values. Specifically, sweep `alpha` from 1E-2 to 1E10 (each step is an order of magnitude jump).

In [38]:
for i in range(-2,11):
    al = 10**i
    print('alpha:',al)
    clf = Ridge(alpha=al)
    clf.fit(Xs_poly_train, y_train)
    y_clf_train = clf.predict(Xs_poly_train)
    y_clf_val = clf.predict(Xs_poly_val)
    print('RMSE for training data:', rmse(y_clf_train,y_train))
    print('RMSE for validation data:',rmse(y_clf_val,y_val))
    print('----------------')
# clf = Ridge(alpha=1.0)
# clf.fit(Xs_poly_train, y_train)
# print(clf.score(Xs_poly_train, y_train))
# print(clf.score(Xs_poly_val, y_val))

alpha: 0.01
RMSE for training data: 321.0124089657283
RMSE for validation data: 10919504.125894751
----------------
alpha: 0.1
RMSE for training data: 3199.349646904824
RMSE for validation data: 10916171.329033157
----------------
alpha: 1
RMSE for training data: 30988.51834150589
RMSE for validation data: 10884651.018828351
----------------
alpha: 10
RMSE for training data: 248080.36964934872
RMSE for validation data: 10659707.103879165
----------------
alpha: 100
RMSE for training data: 1196923.6228686057
RMSE for validation data: 9484394.220119663
----------------
alpha: 1000
RMSE for training data: 3609078.2596920743
RMSE for validation data: 5633182.875138328
----------------
alpha: 10000
RMSE for training data: 7712389.432217537
RMSE for validation data: 4521905.5835024
----------------
alpha: 100000
RMSE for training data: 14600498.52034878
RMSE for validation data: 10502863.775565725
----------------
alpha: 1000000
RMSE for training data: 20036025.005990032
RMSE for validation 